In [1]:
import mwpf_rational as mwpf
mwpf.Visualizer.embed()

# Paper Logic Flow

The goal of the MWPF paper is to explain the mwpf algorithm in a way that people can easily understand.

## 1. explain decoding hypergraph and its parity matrix

The requriements of the example are:
1. only a few number of vertices and edges
2. show degree-1, 2, and 3 edges
3. has some degree of freedom (short loops)
4. suitable for explaining both simple and complicated decoding process

In [2]:
import math
class Example:
    def __init__(self, syndrome: list[int] = [4]):
        self.vertex_num = 5
        self.positions = [mwpf.VisualizePosition(i, j, 0) for (i, j) in [(-1, 0), (0, -1), (1, 0), (0, 1), (1, 1)]]
        self.weighted_edges = [
            mwpf.HyperEdge([0], 1),
            mwpf.HyperEdge([0, 3], 1),
            mwpf.HyperEdge([0, 1], 1),
            mwpf.HyperEdge([1, 2], 1),
            mwpf.HyperEdge([2, 3, 4], 1),
        ]
        self.syndrome = list(syndrome)
        self.incident_edges = [[] for _ in range(self.vertex_num)]
        for edge_index, hyperedge in enumerate(self.weighted_edges):
            for vertex_index in hyperedge.vertices:
                self.incident_edges[vertex_index].append(edge_index)
        self.defects = frozenset(syndrome)
    def initializer(self):
        return mwpf.SolverInitializer(self.vertex_num, self.weighted_edges)
    def show(self):
        visualizer = mwpf.Visualizer(positions=self.positions)
        initializer = self.initializer()
        solver = mwpf.Solver(initializer)
        solver.load_syndrome(mwpf.SyndromePattern(self.syndrome))
        visualizer.snapshot("hypergraph", solver)
        visualizer.show()        

In [3]:
example = Example()
example.show()

In [9]:
# print a colored version
visualizer = mwpf.Visualizer(positions=example.positions)
initializer = example.initializer()
solver = mwpf.Solver(initializer)
for edge_index in range(len(example.weighted_edges)):
    node = solver.create_node_hair_unchecked({edge_index})
solver.grow(mwpf.Rational(1))
solver.load_syndrome(mwpf.SyndromePattern(example.syndrome))
visualizer.snapshot("hypergraph", solver)
visualizer.show()        

The parity matrix of this decoding hypergraph is as follows

In [4]:
matrix = mwpf.BasicMatrix()
initializer = example.initializer()
for vertex_index, edge_indicies in enumerate(example.incident_edges):
    print(vertex_index, edge_indicies)
    matrix.add_constraint(vertex_index, edge_indicies, vertex_index in example.defects)
matrix.show()

0 [0, 1, 2]
1 [2, 3]
2 [3, 4]
3 [1, 4]
4 [4]


## 2. explain transformations on decoding hypergraph and parity matrix: XOR vertex $\Leftrightarrow$ XOR row

## 3. explain the logic flow of the algorithm

- we would like to find feasible primal and dual solutions with a small gap $\sum w_e x_e - \sum y_S$
- This involves two separate steps: increase the dual and decrease the primal
- However, neither of these tasks are easy.
    - For dual problem, there are an exponential number of dual variables, and how to efficiently pick a small portion of them to constitute $\sum y_S$ is challenging.
    - For primal problem, although there is only linear number of variables with the number of physical error locations, finding the minimum-weighted one requires one to search over the exponential space especially with the high degeneracy in quantum error correction.
- We are inspired by the blossom algorithm, which solves the primal and dual problems leveraging each other's information. This is backed by the complementary slackness theorem that states the property of the optimal solution.
    - The primal module uses the dual module information to select solutions: only tight edges can be selected
    - The dual module uses the primal solution space to identify new dual variables, based on the single-hair rule
- More thoughts: since dual can be sometimes suboptimal due to the suboptimality of the disjoint single hair algorithm, it's probably beneficial to slightly break the law of the complementary slackness theorem
    - Explore some more region in the primal graph?
- More thoughts: equivalent decoding graphs
    - Indeed, we showed that there are multiple equivalent decoding hypergraphs, equivalent to each other via vertex manipulations. However, not all of them are created equal: some of them are sparse and some of them are dense. It would be much harder to reduce the primal-dual gap given a (equivalent) dense graph compared to a sparse graph.

Note:

Single-hair algorithm could be divided into three levels:
1. ensure that for each non-zero dual variable, there exists one solution using the tight edges that only uses a single hair
2. ensure that for each non-zero dual variable, for each of its tight hair corresponds to a single hair solution of this dual variable
3. ensure that there exists a tight hair solution that is single-hair solution for all non-zero dual variables

If an algorithm reaches level 3, it is an optimal solution.